In [13]:
import json
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import T5Tokenizer, MT5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup

# Google's Official Preprocess Codes
# https://github.com/google-research/language/blob/master/language/totto/baseline_preprocessing/preprocess_utils.py
from preprocess_utils import get_highlighted_subtable, linearize_subtable

In [4]:
# Train Config
device=torch.device('cpu')
lr=3e-1
batch_size=8 # 4(max 6) for 't5-large' and make 'accumulation_steps' larger
accumulation_steps=1
epochs=20

# Prompt Config
prompt_len=100
hidden_dim=768

In [22]:
# Pre-Trained T5 Tokenizer
tokenizer=T5Tokenizer.from_pretrained('google/mt5-base')
# Add Special Tokens: Table Tags
tokenizer.add_special_tokens({
    'additional_special_tokens': [
        '|',
        ':',
    ]
})
# Pre-Trained T5 Model
pretrained = MT5ForConditionalGeneration.from_pretrained('google/mt5-base').to(device)
# Resize PLM's Embedding Layer
pretrained.resize_token_embeddings(len(tokenizer))
# Freeze LM
for param in pretrained.parameters():
    param.requires_grad=False

In [23]:
class WebNLGDataset(Dataset):
    
    def __init__(self, tokenizer, raw_path='../webnlg_data/release_v3.0/ru', language='en', data_path='../webnlg_data/preprocessed', split='train'):
        
        if not os.path.exists(f'{data_path}/{split}.json'):
            b = Benchmark()
            files = select_files(raw_path)
            b.fill_benchmark(files)
            b.b2json(data_path, f'{split}.json')
        
        with open(f'{data_path}/{split}.json', 'r') as f:
            dataset = json.load(f)
            entries = dataset['entries']

        full_rela_lst = []
        full_src_lst = []
        full_tgt_lst = []
        for i, entry in enumerate(entries):
            sents = entry[str(i + 1)]['lexicalisations']
            triples = entry[str(i + 1)]['modifiedtripleset']
            
            rela_lst = []
            temp_triples = ''
            for j, tripleset in enumerate(triples):
                subj, rela, obj = tripleset['subject'], tripleset['property'], tripleset['object']
                rela_lst.append(rela)
                temp_triples += ' | '
                temp_triples += '{} : {} : {}'.format(subj, rela, obj)

            for sent in sents:
                if sent["lang"] == language:
                    full_tgt_lst.append(sent["lex"])
                    full_src_lst.append(temp_triples)
                    full_rela_lst.append(rela_lst)
                    if split == 'dev':
                        break
            
        assert len(full_rela_lst) == len(full_src_lst)
        assert len(full_rela_lst) == len(full_tgt_lst)

        self.examples = []
        self.targets = []
        for src, tgt in zip(full_src_lst, full_tgt_lst):
            print(src)
            src = tokenizer.encode(src)
            if len(src)>512:
                # Truncate
                encoded = src[:511] + [tokenizer.eos_token_id]
            self.examples.append(src)
    
            print(tgt)
            tgt = tokenizer.encode(tgt)
            self.targets.append(tgt)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx], self.targets[idx]


In [24]:
class ToTToDataset(Dataset):
    def __init__(self, path_data, tokenizer):
        #
        self.data=[]
        self.label=[]
        
        # Load Dataset
        with open(path_data, 'r') as f:
            dataset=f.read().splitlines()
            f.close()
            
        for _data in dataset:
            data=json.loads(_data)
            
            # Preprocess
            subtable=get_highlighted_subtable(table=data['table'], cell_indices=data['highlighted_cells'], with_heuristic_headers=True)
            cells_linearized=linearize_subtable(
                subtable=subtable,
                table_page_title=data['table_page_title'],
                table_section_title=data['table_section_title']
            )
            
            # Encode
            print(cells_linearized)
            encoded=tokenizer.encode(cells_linearized)
            if len(encoded)>512:
                # Truncate
                encoded=encoded[:511]+[tokenizer.eos_token_id]
            self.data.append(encoded)
            print(data['sentence_annotations'][0]['final_sentence'])
            self.label.append(tokenizer.encode(data['sentence_annotations'][0]['final_sentence']))
            
        print(len(self.data), 'datas')
        print(len(self.label), 'labels')
        
    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]
        
    def __len__(self):
        return len(self.data)

In [27]:
def collate_fn(batch):
    """
    Same Sequence Length on Same Batch
    """
    max_len_data=0
    max_len_label=0
    for data, label in batch:
        if len(data)>max_len_data: max_len_data=len(data)
        if len(label)>max_len_label: max_len_label=len(label)
            
    datas=[]
    attn_masks=[]
    labels=[]
    for data, label in batch:
        data.extend([tokenizer.pad_token_id]*(max_len_data-len(data)))
        datas.append(data)
        
        attn_mask=[int(e!=tokenizer.pad_token_id) for e in data]
        attn_masks.append(attn_mask)
        
        label.extend([-100]*(max_len_label-len(label)))
        labels.append(label)
        
    return torch.tensor(datas), torch.tensor(attn_masks), torch.tensor(labels)

In [28]:
dataset_train=WebNLGDataset(tokenizer=tokenizer)
dataloader_train=DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

 | Aarhus_Airport : cityServed : "Aarhus, Denmark"
The Aarhus is the airport of Aarhus, Denmark.
 | Aarhus_Airport : cityServed : "Aarhus, Denmark"
Aarhus Airport serves the city of Aarhus, Denmark.
 | Aarhus_Airport : cityServed : Aarhus
Aarhus airport serves the city of Aarhus.
 | Aarhus_Airport : elevationAboveTheSeaLevel : 25.0
Aarhus Airport is 25 metres above sea level.
 | Aarhus_Airport : elevationAboveTheSeaLevel : 25.0
Aarhus airport is at an elevation of 25 metres above seal level.
 | Aarhus_Airport : elevationAboveTheSeaLevel : 25.0
Aarhus Airport is 25.0 metres above the sea level.
 | Aarhus_Airport : location : Tirstrup
Aarhus Airport is located in Tirstrup.
 | Aarhus_Airport : location : Tirstrup
The location of Aarhus Airport is Tirstrup.
 | Aarhus_Airport : operatingOrganisation : "Aarhus Lufthavn A/S"
Aarhus Airport is operated by Aarhus Lufthavn A/S.
 | Aarhus_Airport : operatingOrganisation : "Aarhus Lufthavn A/S"
Aarhus Lufthavn A/S is the operation organisation of 

 | Ashgabat_International_Airport : runwayName : "12R/30L"
Ashgabat International Airport has the runway name 12R/30L.
 | Ashgabat_International_Airport : runwayName : "12R/30L"
Ashgabat International Airport has a runway named 12R/30L.
 | Athens : country : Greece
Athens is located in the country of Greece.
 | Athens : country : Greece
Athens is a city in Greece.
 | Athens_International_Airport : cityServed : Athens
The Athens International Airport serves the city of Athens.
 | Athens_International_Airport : cityServed : Athens
Athens International Airport serves the city of Athens.
 | Athens_International_Airport : elevationAboveTheSeaLevelInMetres : 94
Athens International Airport is 94 metres above sea level.
 | Athens_International_Airport : elevationAboveTheSeaLevelInMetres : 94
The elevation above the sea level (in metres) of Athens International Airport is 94.
 | Athens_International_Airport : runwayLength : 3800.0
The runway length at Athens International Airport is 3,800.
 | 

 | Apollo_12 : backupPilot : Alfred_Worden
Alfred Worden was a backup pilot of Apollo 12.
 | Apollo_12 : backupPilot : Alfred_Worden
Apollo 12's backup pilot was Alfred Worden.
 | Apollo_12 : backupPilot : Alfred_Worden
Alfred Worden was the backup pilot of Apollo 12.
 | Apollo_12 : commander : David_Scott
Apollo 12 was commanded by David Scott.
 | Apollo_12 : commander : David_Scott
David Scott was the commander of Apollo 12.
 | Apollo_12 : operator : NASA
The operator of Apollo 12 was NASA.
 | Apollo_12 : operator : NASA
The Apollo 12 operator is NASA.
 | Apollo_12 : operator : NASA
NASA operated Apollo 12.
 | Apollo_12 : operator : NASA
The Apollo 12 mission was operated by NASA.
 | Apollo_12 : operator : NASA
Apollo 12 is operated by NASA.
 | Apollo_8 : commander : Frank_Borman
Frank Borman was an Apollo 8 Commander.
 | Apollo_8 : commander : Frank_Borman
Frank Borman was the Apollo 8 commander.
 | Apollo_8 : commander : Frank_Borman
The Apollo 8 commander was Frank Borman.
 | Apol

 | Adisham_Hall : address : "St. Benedict's Monastery, Adisham, Haputhale, Sri Lanka"
The address of Adisham Hall is, St Benedict's Monastery, Adisham, Haputhale, Sri Lanka.
 | Adisham_Hall : architecturalStyle : "Tudor and Jacabian"
Adisham Halls Architectural style`is Tudor and Jacabian.
 | Adisham_Hall : architecturalStyle : "Tudor and Jacabian"
The architecture style of Adisham Hall is Tudor and Jacabian.
 | Adisham_Hall : architecturalStyle : "Tudor and Jacabian"
Adisham Hall is in the architectural style of "Tudor and Jacobean".
 | Adisham_Hall : architecturalStyle : Tudor_Revival_architecture
Adisham Hall has the Tudor Revival architectural style.
 | Adisham_Hall : architecturalStyle : Tudor_Revival_architecture
Adisham Hall has the architectural style 'Tudor Revival'.
 | Adisham_Hall : architecturalStyle : Tudor_Revival_architecture
The Adisham Hall's style of architecture is Tudor Revival.
 | Adisham_Hall : buildingStartDate : "1927"
Construction of Adisham Hall began in 1927.

 | (19255)_1994_VK8 : escapeVelocity : 0.0925 (kilometrePerSeconds)
(19255) 1994 VK8 has an escape velocity of 0.0925 km/s.
 | (19255)_1994_VK8 : mass : 5.6 (kilograms)
19255 1994 VK8 has a mass of 5.6 kgs.
 | (19255)_1994_VK8 : mass : 5.6 (kilograms)
(19255) 1994 VK8 has a mass of 5.6 kilograms.
 | (19255)_1994_VK8 : mass : 5.6 (kilograms)
19255 1994 VK8 has a mass of 5.6kg.
 | (19255)_1994_VK8 : orbitalPeriod : 8788850000.0
(19255) 1994 VK8 has an orbital period of 8788850000.0.
 | (19255)_1994_VK8 : periapsis : 6155910000000.0
(19255) 1994 VK8 has a periapsis of 6155910000000.0.
 | (19255)_1994_VK8 : periapsis : 6155910000000.0
The periapsis of (19255) 1994 VK8 is 6155910000000.0.
 | (19255)_1994_VK8 : temperature : 43.0 (kelvins)
(19255) 1994 VK8 has a temperature of 43.0 kelvins.
 | (19255)_1994_VK8 : temperature : 43.0 (kelvins)
The temperature of 19255 1994 VK8 is 43 kelvins.
 | (29075)_1950_DA : density : 3.5 (kilograms)
(29075) 1950 DA has a density of 3.5 kilograms.
 | (29075

...

Alan Shepard (died on 1998-07-21 in California) graduated from NWC in 1957.
 | Alan_Shepard : almaMater : "NWC, M.A. 1957" | Alan_Shepard : deathPlace : California | Alan_Shepard : deathDate : "1998-07-21"
Alan Shepard, who attended NWC, M.A. in 1957, died in California on July 21, 1998.
 | Alan_Shepard : birthPlace : New_Hampshire | Alan_Shepard : selectedByNasa : 1959 | Alan_Shepard : birthDate : "1923-11-18"
Alan Shepard who was born on Nov 18, 1923 in New Hampshire was selected by NASA in 1959.
 | Alan_Shepard : birthPlace : New_Hampshire | Alan_Shepard : selectedByNasa : 1959 | Alan_Shepard : birthDate : "1923-11-18"
Alan Shepard was born on November 18th, 1923 in New Hampshire and was selected by NASA in 1959.
 | Alan_Shepard : birthPlace : New_Hampshire | Alan_Shepard : selectedByNasa : 1959 | Alan_Shepard : birthDate : "1923-11-18"
Alan Shepard was born in New Hampshire, chosen by NASA in 1959 and was born on the 18th of November 1923.
 | Alan_Shepard : birthPlace : New_Hampshi

108 St Georges Terrace has a floor area of 39599.0 square metres and is located in Perth, Australia.
 | 108_St_Georges_Terrace : location : Perth | Perth : country : Australia | 108_St_Georges_Terrace : floorCount : 50
108 St. Georges Terrace boasts 50 floors and is located in Perth, Australia.
 | 108_St_Georges_Terrace : location : Perth | Perth : country : Australia | 108_St_Georges_Terrace : floorCount : 50
108 St. Georges Terrace in Perth, Australia has 50 floors.
 | 108_St_Georges_Terrace : location : Perth | Perth : country : Australia | 108_St_Georges_Terrace : floorCount : 50
The 108 St. Georges Terrace in Perth, Australia, has a floor count of 50.
 | 11_Diagonal_Street : floorCount : 20 | 11_Diagonal_Street : architect : Helmut_Jahn | 11_Diagonal_Street : completionDate : 1983
11 Diagonal Street, with 20 floors, was designed by Helmut Jahn and completed in 1983.
 | 11_Diagonal_Street : floorCount : 20 | 11_Diagonal_Street : architect : Helmut_Jahn | 11_Diagonal_Street : comple

Adare Manor was completed in 1862, opened in 1700 and was designed by James Pain and George Richard Pain.
 | Adare_Manor : architect : "James Pain and George Richard Pain," | Adare_Manor : completionDate : 1862 | Adare_Manor : owner : J._P._McManus
James Pain and George Richard Pain are the architects of Adare Manor, which was completed in 1862, and is owned by JP McManus.
 | Adare_Manor : architect : "James Pain and George Richard Pain," | Adare_Manor : completionDate : 1862 | Adare_Manor : owner : J._P._McManus
The architects for Adare Manor completed in 1862 were James Pain and George Richard Pain. The Manor is owned by JP McManus.
 | Adare_Manor : architect : "James Pain and George Richard Pain," | Adare_Manor : completionDate : 1862 | Adare_Manor : owner : J._P._McManus
Adare Manor, owned by J. P. McManus, was designed by James and George Richard Pain and its construction was completed in 1862.
 | Adare_Manor : architect : Lewis_Nockalls_Cottingham | Adare_Manor : completionDate :

 | (19255)_1994_VK8 : density : 2.0 (gramPerCubicCentimetres) | (19255)_1994_VK8 : escapeVelocity : 0.0925 (kilometrePerSeconds) | (19255)_1994_VK8 : apoapsis : 6603633000.0 (kilometres)
The celestial body known as (19255) 1994 VK8 has a density of 2 grams per cubic centimetres,an escape velocity of 0.0925 km/s and its apoapsis is 6603633000.0 kilometres.
 | (19255)_1994_VK8 : density : 2.0 (gramPerCubicCentimetres) | (19255)_1994_VK8 : escapeVelocity : 0.0925 (kilometrePerSeconds) | (19255)_1994_VK8 : apoapsis : 6603633000.0 (kilometres)
(19255) 1994 VK8 has a density of 2.0 grams per cubic centimetre, an escape velocity of 0.0925 km/s, and an apoapsis of 6603633000.0 kilometres.
 | (19255)_1994_VK8 : density : 2.0 (gramPerCubicCentimetres) | (19255)_1994_VK8 : escapeVelocity : 0.0925 (kilometrePerSeconds) | (19255)_1994_VK8 : apoapsis : 6603633000.0 (kilometres)
(19255) 1994 VK8 has a density of 2.0 grams per cu cm, an escape velocity of 0.0925 km per sec and an apoapsis of 660363300

KeyboardInterrupt: 

In [ ]:
class PromptTuning(nn.Module):
    """
    """
    def __init__(self, pretrained_config, prompt_len=20, hidden_dim=256):
        super().__init__()
        
        # Config of Pre-Trained LM
        self.pretrained_config=pretrained_config
        
        # torch.tensor([0, 1, 2, .. , prompt_len-1])
        self.pre_prompt=torch.arange(prompt_len)
        # Embedding
        self.embd=nn.Embedding(num_embeddings=prompt_len, embedding_dim=pretrained_config.d_model)
        # Reparameterization
        self.reparam=nn.Sequential(
            nn.Linear(pretrained_config.d_model, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, pretrained_config.d_model)
        )
        
    def forward(self, batch_size, device):
        # Shape: batch_size, prompt_len
        prompt=self.pre_prompt.unsqueeze(0).expand(batch_size, -1).to(device)
        # Shape: batch_size, prompt_len, d_model
        prompt=self.embd(prompt)
        # Shape: batch_size, prompt_len, d_model
        prompt=self.reparam(prompt)
        
        return prompt

In [29]:
# Model: Prompt Tuning
model=PromptTuning(pretrained_config=pretrained.config, prompt_len=prompt_len, hidden_dim=hidden_dim)

# Optim, Scheduler
optimizer=AdamW(model.parameters(), lr=lr)
# NO Warm-Up
scheduler=get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=int(epochs*len(dataset_train)/(accumulation_steps*batch_size))
)

# TensorBoard: Logging
writer=SummaryWriter()
step_global=0

for epoch in range(epochs):
    # Train Phase
    model.train()
    model.to(device)
    
    loss_train=0
    optimizer.zero_grad()
    
    for step, (data, attn_mask, label) in enumerate(dataloader_train):
        data=data.to(device)
        attn_mask=attn_mask.to(device)
        label=label.to(device)
        
        prompt=model(batch_size=data.shape[0] , device=device)
        outputs=pretrained(input_ids=data, attention_mask=attn_mask, labels=label, prompt=prompt)
        
        loss=outputs[0]/accumulation_steps
        loss.backward()
        
        loss_train+=loss.item()
        
        if (step+1)%accumulation_steps==0:
            step_global+=1
            
            # TensorBoard
            writer.add_scalar(
                f'loss_train/MT5-base_Prompt-Tuning_prompt-len{prompt_len}_hidden-dim{hidden_dim}_lr{lr}_batch{int(accumulation_steps*batch_size)}_epoch{epochs}',
                loss_train,
                step_global
            )
            # Console
            if step_global%1000==0:
                print(f'epoch {epoch+1} step {step_global} loss_train {loss_train:.4f}')
            # Set Loss to 0
            loss_train=0
            
            optimizer.step()
            scheduler.step()
            
            optimizer.zero_grad()
            
    # Save Model
    model.to(torch.device('cpu'))
    torch.save(model, f'../model/MT5-base_Prompt-Tuning_prompt-len{prompt_len}_hidden-dim{hidden_dim}_lr{lr}_batch{int(accumulation_steps*batch_size)}_epoch{epoch+1}of{epochs}.pt')

NameError: name 'PromptTuning' is not defined